In [ ]:
import logging

import numpy as np
import optimistix as optx

from atmodeller import (
    InteriorAtmosphere,
    Planet,
    SolverParameters,
    Species,
    SpeciesCollection,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# K2-18b gas dwarf models from Bower et al. (2025)

The code blocks below must always be run, but then you can preferentially run only the models for ideal or real gases with or without solubility.

Parameters for the simulations

In [ ]:
number_of_realisations = 10000
surface_temperature = 2000.0  # Must be float
planet_mass = 5.15211e25
surface_radius = 1.6647e7
mantle_melt_fraction = 1.0  # 0.1
log10_max_oceans = 3

np.random.seed(0)
log10_number_oceans = np.random.uniform(0, log10_max_oceans, number_of_realisations)
log10_ch_ratios = np.random.uniform(-2, 1, number_of_realisations)
log10_nh_ratios = np.random.uniform(-4, -1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
c_kg = h_kg * 10**log10_ch_ratios
n_kg = h_kg * 10**log10_nh_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
    "N": n_kg,
}

In [ ]:
k218b = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

## Ideal gas no solubility

In [ ]:
H2O_g = Species.create_gas("H2O_g")
H2_g = Species.create_gas("H2_g")
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g")
CO2_g = Species.create_gas("CO2_g")
CH4_g = Species.create_gas("CH4_g")
N2_g = Species.create_gas("N2_g")
NH3_g = Species.create_gas("NH3_g")

species = SpeciesCollection((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g))

In [ ]:
model_ideal_nosol = InteriorAtmosphere(species)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Models all solve first time so we reduce multistart for speed
solver_parameters = SolverParameters(multistart=1)
model_ideal_nosol.solve(
    planet=k218b,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_nosol = model_ideal_nosol.output

# Quick look at the solution
# solution_ideal_nosol = output_ideal_nosol.quick_look()

# Get complete solution as a dictionary
# solution_ideal_nosol_asdict = output_ideal_nosol.asdict()
# logger.info(solution_ideal_nosol_asdict)

# Write the complete solution to Excel
output_ideal_nosol.to_excel("k218b_ideal_nosol")

# Write the data to a pickle file with dataframes
output_ideal_nosol.to_pickle("k218b_ideal_nosol")

## Ideal gas with solubility

In [ ]:
solubility_models = get_solubility_models()

In [ ]:
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2_g", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2_g", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4_g", solubility=solubility_models["CH4_basalt_ardia13"])
# N2_g = Species.create_gas("N2_g", solubility=solubility_models["N2_basalt_libourel03"])
# Dasgupta law might be preferred for higher pressure regime
N2_g = Species.create_gas("N2_g", solubility=solubility_models["N2_basalt_dasgupta22"])
NH3_g = Species.create_gas("NH3_g")

species = SpeciesCollection((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g))

In [ ]:
model_ideal_withsol = InteriorAtmosphere(species)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

solver_parameters = SolverParameters(multistart=1)
model_ideal_withsol.solve(
    planet=k218b,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_withsol = model_ideal_withsol.output

# Quick look at the solution
# solution_ideal_withsol = output_ideal_withsol.quick_look()

# Get complete solution as a dictionary
# solution_ideal_withsol_asdict = output_ideal_withsol.asdict()
# logger.info(solution_ideal_withsol_asdict)

# Write the complete solution to Excel
output_ideal_withsol.to_excel("k218b_ideal_withsol")

# Write the data to a pickle file with dataframes
output_ideal_withsol.to_pickle("k218b_ideal_withsol")

## Real gas no solubility

In [ ]:
eos_models = get_eos_models()

In [ ]:
H2O_g = Species.create_gas(
    "H2O_g",
    activity=eos_models["H2O_cork_holland98"],
)
H2_g = Species.create_gas("H2_g", activity=eos_models["H2_shi92"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas(
    "CO_g",
    activity=eos_models["CO_cork_cs_holland91"],
)
CO2_g = Species.create_gas(
    "CO2_g",
    activity=eos_models["CO2_cork_holland98"],
)
CH4_g = Species.create_gas(
    "CH4_g",
    activity=eos_models["CH4_cork_cs_holland91"],
)
N2_g = Species.create_gas(
    "N2_g",
    activity=eos_models["N2_cork_cs_holland91"],
)
NH3_g = Species.create_gas("NH3_g")

species = SpeciesCollection((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g))

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver, multistart=1)

model_real_nosol = InteriorAtmosphere(species)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# For the initial solution we can use the estimate for ideal without solubilty
initial_log_number_density = output_ideal_nosol.log_number_density
initial_log_stability = output_ideal_nosol.log_stability

model_real_nosol.solve(
    planet=k218b,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    solver_parameters=solver_parameters,
)
output_real_nosol = model_real_nosol.output

# Quick look at the solution
# solution_real_nosol = output_real_nosol.quick_look()

# Get complete solution as a dictionary
# solution_real_nosol_asdict = output_real_nosol.asdict()
# logger.info(solution_real_nosol_asdict)

# Write the complete solution to Excel
output_real_nosol.to_excel("k218b_real_nosol")

# Write the data to a pickle file with dataframes
output_real_nosol.to_pickle("k218b_real_nosol")

## Real gas with solubility

In [ ]:
eos_models = get_eos_models()
solubility_models = get_solubility_models()

In [ ]:
H2O_g = Species.create_gas(
    "H2O_g",
    activity=eos_models["H2O_cork_holland98"],
    solubility=solubility_models["H2O_basalt_dixon95"],
)
H2_g = Species.create_gas(
    "H2_g", activity=eos_models["H2_shi92"], solubility=solubility_models["H2_basalt_hirschmann12"]
)
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas(
    "CO_g",
    activity=eos_models["CO_cork_cs_holland91"],
    solubility=solubility_models["CO_basalt_yoshioka19"],
)
CO2_g = Species.create_gas(
    "CO2_g",
    activity=eos_models["CO2_cork_holland98"],
    solubility=solubility_models["CO2_basalt_dixon95"],
)
CH4_g = Species.create_gas(
    "CH4_g",
    activity=eos_models["CH4_cork_cs_holland91"],
    solubility=solubility_models["CH4_basalt_ardia13"],
)
# N2_g = Species.create_gas(
#     "N2_g",
#     activity=eos_models["N2_cork_cs_holland91"],
#     solubility=solubility_models["N2_basalt_libourel03"],
# )
# Dasgupta law might be preferred for higher pressure regime
N2_g = Species.create_gas(
    "N2_g",
    activity=eos_models["N2_cork_cs_holland91"],
    solubility=solubility_models["N2_basalt_dasgupta22"],
)
NH3_g = Species.create_gas("NH3_g")

species = SpeciesCollection((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g))

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver, multistart=1)

model_real_withsol = InteriorAtmosphere(species)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Could use to inform the initial solution, but it is not required
# initial_log_number_density = output_ideal_withsol.log_number_density
# initial_log_stability = output_ideal_withsol.log_stability

model_real_withsol.solve(
    planet=k218b,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    # initial_log_number_density=initial_log_number_density,
    # initial_log_stability=initial_log_stability,
    solver_parameters=solver_parameters,
)
output_real_withsol = model_real_withsol.output

# Quick look at the solution
# solution_real_withsol = output_real_withsol.quick_look()

# Get complete solution as a dictionary
# solution_real_withsol_asdict = output_real_withsol.asdict()
# logger.info(solution_real_withsol_asdict)

# Write the complete solution to Excel
output_real_withsol.to_excel("k218b_real_withsol")

# Write the data to a pickle file with dataframes
output_real_withsol.to_pickle("k218b_real_withsol")